In [231]:
import pandas as pd

In [232]:
df = pd.read_csv('ab_data.csv')

In [233]:
df.head(5)

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [234]:
# TOTAL COUNT OF MISALINED DATA IN THE EXPERIMENT SET
(((df.group =='treatment') & (df.landing_page == 'old_page')) | ((df.group =='control') & (df.landing_page == 'new_page'))).sum()

3893

In [235]:
df_fixed = df[((df.group =='treatment') & (df.landing_page == 'old_page')) | ((df.group =='control') & (df.landing_page == 'new_page')) == False]

In [236]:
df_fixed.groupby(['group']).count()

,user_id,timestamp,landing_page,converted
group,,,,
control,145274,145274,145274,145274
treatment,145311,145311,145311,145311


In [237]:
df_fixed['user_id'].value_counts().head()

773192    2
630732    1
811737    1
797392    1
795345    1
Name: user_id, dtype: int64

In [238]:
df_fixed[df_fixed['user_id'] == 773192]

,user_id,timestamp,group,landing_page,converted
1899,773192,2017-01-09 05:37:58.781806,treatment,new_page,0
2893,773192,2017-01-14 02:55:59.590927,treatment,new_page,0


In [239]:
df_fixed = df_fixed.drop(index=1899)
df_fixed[df_fixed['user_id'] == 773192]

,user_id,timestamp,group,landing_page,converted
2893,773192,2017-01-14 02:55:59.590927,treatment,new_page,0


In [241]:
convertedcount = df_fixed[['group','converted']].groupby(['group']).sum().values

In [242]:
groupcount = df_fixed[['group','converted']].groupby(['group']).count().values

In [243]:
conversion_rate = pd.DataFrame(convertedcount/groupcount*100,columns=['convertion_rate'],index=['control','treatment'])
conversion_rate

,convertion_rate
control,12.038630
treatment,11.880807


array([[0.12039918],
       [0.11891958]])

The hypothesis is something like this based on the PICOT

- Null Hypothesis : The Visitors of website (P - population) who recieve Layout B (Treatment Group) will not have higher conversion rate as compared to the those who recieve Layout A (control Group) at the end of a visit
- Alternate Hypothesis : The Visitors of website (P - population) who recieve Layout B (Treatment Group) will have higher conversion rate as compared to the those who recieve Layout A (control Group) at the end of a visit


##### P - Population of Visitors ,  I - Intervention in the form of Layout A/B, C - Comparison - Treatment group vs Control group, O - outcome - Converstion rate , T - at what time - At the end of a visit.

Translating the above : let converstion rate of old page be Co and new page be Cn

    H0 = Co - Cn >=0
    H1 = Co - Cn <0

In [245]:
# Using the Stats model to do T Test

In [246]:
import statsmodels.api as sm

In [250]:
convert_old = (df_fixed.query('landing_page=="old_page"')['converted']==1).sum()
convert_new = (df_fixed.query('landing_page=="new_page"')['converted']==1).sum()
n_old = (df_fixed['landing_page']=='old_page').sum()
n_new=(df_fixed['landing_page']=='new_page').sum()

convert_old, convert_new, n_old, n_new  

(17489, 17264, 145274, 145310)

In [270]:
z_score, p_value = sm.stats.proportions_ztest([convert_new, convert_old], [n_new, n_old], alternative='smaller')
z_score, p_value

(-1.3109241984234394, 0.09494168724097551)

In [271]:
from scipy.stats import norm
# Tells us how significant our z-score is
print(norm.cdf(z_score))

# for our single-sides test, assumed at 95% confidence level, we calculate: 
print(norm.ppf(0.05))

0.09494168724097551
-1.6448536269514729


Since our z critical is -1.64 and our z stat is .094, Thus we cannot reject the null hypothesis thats Visitors with Layout 2 will not have higher conversion rate.

In [266]:
# using Ttest 

control = df_fixed[df_fixed['group']=='control'].converted
treatment = df_fixed[df_fixed['group']=='treatment'].converted

In [267]:
from statsmodels.stats.weightstats import ttest_ind

In [269]:
ttest_ind(control,treatment,alternative='smaller')

(1.3109235634981533, 0.9050576875062873, 290582.0)

In [274]:
norm.ppf(.05)

(-1.6448536269514729, 0.9050582054934152)

### Thus we do not reject the null hypothesis. So our overall claim that New Layout will have higher conversion rate is not proven by the experiment. 